# Real-Time Illicit Activity Classification: Proposed Blocks & Mempool

### Import Required Libraries

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from web3 import Web3
import psycopg2
import pandas as pd
import joblib
from etherscan import Etherscan
import custom_functions as cf
import sklearn
import datetime
import traceback
from hexbytes import HexBytes

### Connect to Etherscan API

In [ ]:
#API Key
eth = Etherscan('BCEMI7SNDV9VURA8WD16U9HZIQHRNPK59Z')
eth.get_normal_txs_by_address('0x0eD78CCA6cdfa68e08b85C463F29Dd8741Ff06ad'.lower(), 1, 18000000, 'asc')[0]['hash']

### Connect to Quicknode Geth Client (Ethereum Mempool Transactions)

In [ ]:
#Node HTTP URL
quicknode_url = 'https://wandering-proportionate-borough.discover.quiknode.pro/fb6dc3f6a93d9342b7b1f397599fdd5ecc80b370/'
web3 = Web3(Web3.HTTPProvider(quicknode_url))
web3.is_connected()
web3.eth.get_block_number()

### Connect to PostgreSQL Database

In [ ]:
#Ensure Connection to Google Cloud Database
conn = psycopg2.connect(
    host="34.134.242.102",
    database="addresses",
    user="postgres",
    password="AIChain")

cur = conn.cursor()
cur.execute('SELECT version()')
record = cur.fetchone()
record

### Import Ensemble Models (Random Sampling & Coinbase)

In [ ]:
#Import Random Sampling Based Ensemble Model (Currently a Simple Random Forest)
random_sampling_model = joblib.load('v3_ensemble_model_random')

#Import Coinbase Based Ensemble Model
coinbase_model = joblib.load('v3_ensemble_model_coinbase')

### Define Nessesary Variables

In [ ]:
#Define Account-Based Features to Generate
account_feature_list = ['Sent_tnx',
                'Recieved_tnx',
                'Total_Ether_Balance',
                'Max_Value_Received',
                'Min_Value_Received',
                'Total_Ether_Received',
                'Time_Diff_between_first_and_last_(Mins)',
                'Total_Transactions(Including_Tnx_to_Create_Contract)',
                'Avg_Value_Received',
                'Max_Value_Sent',
                'Min_Value_Sent',
                'Total_Ether_Sent',
                'Avg_Value_Sent',
                'Avg_min_between_sent_tnx',
                'Avg_min_between_received_tnx',
                'ERC20_Total_Ether_Received',
                'ERC20_Total_Ether_Sent',
                'ERC20_Min_Value_Rec',
                'ERC20_Max_Value_Rec',
                'ERC20_Min_Value_Sent',
                'ERC20_Max_Value_Sent',
                'Total_ERC20_Tnxs',
                'ERC20_Avg_Time_Between_Sent_Tnx',
                'ERC20_Avg_Time_Between_Rec_Tnx',
                'Unique_Received_From_Addresses',
                'Unique_Sent_To_Addresses',
                'ERC20_Uniq_Sent_Addr',
                'ERC20_Uniq_Rec_Addr',
                'ERC20_Avg_Value_Sent',
                'ERC20_Avg_Value_Rec']

account_feature_list = [i.upper() for i in account_feature_list]

#Define Block Range to Generate Features Over
start_block = 1
end_block = 18000000

### Get Total Number of Transactions in Mempool

In [ ]:
mempool_count = web3.geth.txpool.status()
int(mempool_count['pending'],16)

### View All Mempool Transactions

In [ ]:
mempool = web3.geth.txpool.content()
#mempool

### Define Function To Classify A Single Address Implicated in a Mempool Transaction

In [ ]:
def process_proposed_tx(tx,sender_or_receiver):
    try:
        if sender_or_receiver == 'receiver':
            address = tx['from'].lower()
        if sender_or_receiver == 'sender':
            address = tx['to'].lower()
        whitelist_query = "SELECT flag FROM whitelist WHERE address = " + "'" + address + "'"
        legal_query = "SELECT flag FROM legal_addresses WHERE address = " + "'" + address + "'"
        illicit_query = "SELECT flag FROM illicit_addresses WHERE address = " + "'" + address + "'"
        cur.execute(whitelist_query)
        conn.commit()
        whitelist_record = cur.fetchall()
        #If address isn't in whitelist table, query legal table
        if len(whitelist_record) == 0:
            cur.execute(legal_query)
            conn.commit()
            legal_record = cur.fetchall()
            #If address isn't in legal table, query illicit table
            if len(legal_record) == 0:
                cur.execute(illicit_query)
                conn.commit()
                illicit_record = cur.fetchall()
                #If address isn't in illicit table, generate features & predict
                if len(illicit_record) == 0:
                    etherscan_ouput = cf.automate_feature_generation_single(address, account_feature_list, 1, int(web3.eth.get_block_number()))
                    etherscan_ouput = etherscan_ouput.drop(columns = ['Address'])
                    prediction = coinbase_model.predict(etherscan_ouput)[0]
                    #If the model predicts the address is legal, approve the address & update database
                    if prediction == 0:
                        classification_time = datetime.datetime.now()
                        insert_query = "INSERT INTO legal_addresses (address, flag, timestamp) VALUES (%s,%s,%s)"
                        insert_tuple = (address, 0, classification_time)
                        cur.execute(insert_query,insert_tuple)
                        conn.commit()
                        final_prediction = prediction
                    #If the model predicts the address is illegal, reject the address & update the database
                    if prediction == 1:
                        classification_time = datetime.datetime.now()
                        insert_query = "INSERT INTO illicit_addresses (address, flag, timestamp) VALUES (%s,%s,%s)"
                        insert_tuple = (address, 1, classification_time)
                        cur.execute(insert_query,insert_tuple)
                        conn.commit()
                        final_prediction = prediction
                #If address is in the illicit table, reject the address
                if len(illicit_record) == 1:
                    final_prediction = 1
            #If the address is in the legal table, approve the address
            if len(legal_record) == 1:
                final_prediction = 0
        #If sender address is in the whitelist table, approve the address
        if len(whitelist_record) == 1:
            final_prediction = 0
    except:
        traceback.print_exc()
        conn.rollback()
    return final_prediction

### Define Function to Classify Both Addresses Implicated in a Mempool Transaction

In [ ]:
#Approval For Individial Mempool Transactions to Be Added to Proposed Blocks
def mempool_tx_classifier(tx):
    sender_flag = process_proposed_tx(tx,'sender')
    receiver_flag = process_proposed_tx(tx,'receiver')
    tx_flags = [sender_flag, receiver_flag]
    return tx_flags

transaction = web3.eth.get_transaction('0x8bd2785c93871b8e42b02b0d79150f6657ea44d1ddce8a631f9515981de735d6')
mempool_tx_classifier(transaction)

### Stream Real-Time Mempool Transactions (No Classification)

In [ ]:
from web3.auto import Web3
import asyncio
import json

# enter your web socket node credentials here
# this will allow us to stream transactions
wss = 'wss://wandering-proportionate-borough.discover.quiknode.pro/fb6dc3f6a93d9342b7b1f397599fdd5ecc80b370/'
web3 = Web3(Web3.WebsocketProvider(wss))


# test to see if you are connected to your node
# this will print out True if you are successfully connected to a node
print(web3.is_connected())


def handle_event(event):
    # print the transaction hash
    # print(Web3.to_json(event))

    # use a try / except to have the program continue if there is a bad transaction in the list
    try:
        # remove the quotes in the transaction hash
        transaction = Web3.to_json(event).strip('"')
        # use the transaction hash that we removed the '"' from to get the details of the transaction
        transaction = web3.eth.get_transaction(transaction)
        # print the transaction and its details
        print(transaction)

    except Exception as err:
        # print transactions with errors. Expect to see transactions people submitted with errors 
        print(f'error: {err}')


async def log_loop(event_filter, poll_interval):
    while True:
        for event in event_filter.get_new_entries():
            handle_event(event)
        await asyncio.sleep(poll_interval)


def main():
    # filter for pending transactions
    tx_filter = web3.eth.filter('pending')
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(
            asyncio.gather(
                log_loop(tx_filter, 2)))
    finally:
        loop.close()


if __name__ == '__main__':
    main()

### Preform Real-Time Illicit Activity Detection on Mempool Transactions

In [ ]:
from web3.auto import Web3
import asyncio
import json

wss = 'wss://wandering-proportionate-borough.discover.quiknode.pro/fb6dc3f6a93d9342b7b1f397599fdd5ecc80b370/'
web3 = Web3(Web3.WebsocketProvider(wss))
print(web3.is_connected())

def handle_event(event):
    try:
        transaction = Web3.to_json(event).strip('"')
        transaction = web3.eth.get_transaction(transaction)
        output = mempool_tx_classifier(transaction)
        print('Tx Hash ' + str(transaction['hash'].hex()) + ' has been processed.')
        if (output[0] == 0) & (output[1] == 0):
            print('Both addresses implicated were deemed to be legal. We approve the transaction for block inclusion.')
        if (output[0] == 1) & (output[1] == 0):
            print('The address of the sender was deemed to be illicit. We recommend against block inclusion for this transaction.')
        if (output[0] == 0) & (output[1] == 1):
            print('The address of the receiver was deemed to be illicit. We recommend against block inclusion for this transaction.')
        if (output[0] == 1) & (output[1] == 1):
            print('Both addresses implicated were deemed to be illicit. We recommend against block inclusion for this transaction.')
            

    except Exception as err:
        print(f'error: {err}')


async def log_loop(event_filter, poll_interval):
    while True:
        for event in event_filter.get_new_entries():
            handle_event(event)
        await asyncio.sleep(poll_interval)


def main():
    # filter for pending transactions
    tx_filter = web3.eth.filter('pending')
    loop = asyncio.get_event_loop()
    loop.run_until_complete(asyncio.gather(log_loop(tx_filter, 2)))

if __name__ == '__main__':
    main()